In [1]:
%%time
import pandas as pd
from pathlib import Path
import sqlalchemy
import xlrd
import numpy as np
import pymysql.cursors

CPU times: user 396 ms, sys: 60 ms, total: 456 ms
Wall time: 631 ms


# This is a guide to help convert the Zillow Transaction data to individual CSVs

# 1 Get the data from the Zillow Files

* We need to get the Main table from the Assessor Data, and then the Main and PropertyInfo table. 
    * We can convert the Main.txt files for both Assessment and Transaction files in "Trans and Asmt Data to Database.ipynb" (USE THIS FOR PROPERTY INFO AS WELL)
    * ~~The propert info can be done with "Property Info Data to Database.ipynb"~~
    
Running this will result in a lot of different .db files. These are sqlite3 db files, you can interact with them by using SQL and running 'sqlite3' in terminal in the directory where your files are located

# 2 Join the databases

To link the Assessor data to Transaction data, we need to do a lot of joins. 

From the Zillow Documentation:

> To match events to property attribute information in assessor tables, first join the Main table to PropertyInfo using TransID. Use the ImportParcelID in the PropertyInfo table to link to the property in assessor tables. Use the ImportParcelID to match previous sales or other events on the same property. (TableDescriptions_2016-01.xsls)

So we need to join Transaction to Property info using "TransId", then that resulting table to the Assessor Data using the "ImportParcelID" collumn. We have to do this in terminal because we do not have enough RAM to do this in pandas.


In Terminal (Example is for CT). Navigate to correct folder using "cd" and then type sqlite3 hit enter, it will open up a prompt:
```
attach 'CT_trans.db' as trans;
attach 'CT_property.db' as prop;
CREATE TABLE prop.joined AS SELECT * from trans.fips a inner join prop.PropertyInfo b on a.TransId = b.TransId;
```
This leaves you with a table called "joined" in 'CT_property.db', with the Transaction PropertyInfo and Main joined.

Where "CT_trans.db" is the transaction database for CT and "CT_property.db" is the property database for CT.
Please check the tables in the second line, to make sure that each database has the correct table that you are trying to join. Also, I would highly recommend filtering by FIPS before this step.

Then to join to Asmt:
```
attach 'CT_asmt.db' as asmt;
attach 'CT_property.db' as prop;
CREATE TABLE asmt.combined AS SELECT * from asmt.fips a inner join prop.joined b on a.ImportParcelID = b.ImportParcelID;
```
Now you have a table called "combined" in "CT_asmt.db", this has the Transaction and Assesment Data joined!!!!

# 3 Break into different CSVs for each Unique FIPS

[Look at this tutorial for how to export a CSV from sqlite3 terminal](http://www.sqlitetutorial.net/sqlite-export-csv/)

Here we are going to do something very annoying, we are going to create the individual CSVs by hand. I tried to write scripts for this (and spent a great deal of time doing so) but was unable to do it :(

Once you have the joined DBs, we can split these up into individual DBs for each fips and then convert each db into a CSV. I know this is a pain.

Use the "Joined DB to FIPS CSV" notebook to generate all of this code, then you just have to copy and paste it in to terminal and wait for it to finish.

To make each indivudial Database copy and paste the "sqlite3 <state><fips>.db" in terminal, then type .database, then .quit. This will create each individual database.
    
Then we want to run one instance of 



**Q**: I got "Error: too many attached databases - max 10" 

**A**: L close the sqlite3 instance and relaunch
